# PC3 **Gráfica**

## Integrantes
- Chodury Gómez, Junal
- Llanos Rosadio, José
- Zapata Inga, Janio

# Introducción
Este  cuaderno comparará dos formas diferentes de entrenar una red neuronal:

Red A: Un modelo que primero aprende a reconocer números (0-9) y luego intenta usar ese conocimiento para reconocer formas geométricas.
Red B: Un modelo que aprende desde cero a reconocer formas geométricas.

## Importando dependencias


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import image_dataset_from_directory

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

: 

## Paso 1: Cargar del Dataset
En este ocasión elaboramos un dataset de 8 imágenes dibujadas a mano (triángulo, rectángulo, rombo, X, círculo, semicírculo, estrella, cuadrado)

Primero creamos una pequeña aplicación web, en dónde se puede dibujar  figuras en un "lienzo" y enviarlas a un bucket en supabase.
- https://dibujos-figuras-entrenamiento.netlify.app/

Luego de haber dibujado suficiente, las descargamos todas y las subimos a un repositorio de github
- https://github.com/JunalChowdhuryG/Mi_Dataset

Para que sean compatibles con MNIST, serán de  tamaño  `(28, 28)` pixeles y el modo de color `grayscale`

In [ ]:
!rm -rf Mi_Dataset
!git clone https://github.com/JunalChowdhuryG/Mi_Dataset.git

In [ ]:
# --- Configuración del Experimento ---
DATASET_PATH = 'Mi_Dataset' # Nuestro dataset :)
NUM_CLASSES_CUSTOM = 8      # Son 8 formas
IMG_SIZE = (28, 28)
COLOR_MODE = 'grayscale'
CHANNELS = 1
INPUT_SHAPE = (28, 28, 1)
SEED = 42                   # Para reproducibilidad
BATCH_SIZE = 32             # Batch size para cargar el dataset

print(f"Cargando dataset desde: {DATASET_PATH}")

# Cargando imágenes desde carpetas
full_dataset = image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred',
    label_mode='int',
    color_mode=COLOR_MODE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
    batch_size=BATCH_SIZE
)

# Obtenemos los nombres de las clases (8 carpetas)
class_names = full_dataset.class_names
print(f"Clases encontradas ({len(class_names)}): {class_names}")

# Mapeo de etiquetas
# class_names_map = {0: 'circulo', 1: 'cuadrado', ...}
class_names_map = {i: name for i, name in enumerate(class_names)}
print("Mapeo de etiquetas:", class_names_map)

In [ ]:
# --- Convertir datos ---
# Convertir el tf.data.Dataset a arrays de NumPy (X, y)
def dataset_to_numpy(dataset):
    images = []
    labels = []
    for img_batch, label_batch in dataset.unbatch().batch(1):
        images.append(img_batch.numpy())
        labels.append(label_batch.numpy())

    # Apilamos todo en un gran array
    # (None, 28, 28, 1) -> (TotalImgs, 28, 28, 1)
    X = np.vstack(images)
    y = np.vstack(labels).flatten() # (TotalImgs,)
    return X, y

print("Convirtiendo dataset a formato NumPy...")
X_custom, y_custom = dataset_to_numpy(full_dataset)

# --- Normalización ---
X_custom = X_custom / 255.0

# X (Total, 28, 28, 1) -> X_raw_like (Total, 28, 28)
X_raw_like = X_custom.reshape(X_custom.shape[0], 28, 28)

print(f"Forma X (NumPy): {X_raw_like.shape}")
print(f"Forma y (NumPy): {y_custom.shape}")

In [ ]:
# --- Visualización  ---
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    # Elegimos 25 índices aleatorios
    idx = np.random.choice(X_raw_like.shape[0], 1)[0]

    # Usamos el class_names_map que creamos
    plt.title('id:{} val:{}'.format(idx, class_names_map[y_custom[idx]] ))

    # Usamos X_raw_like[idx] que es (28, 28)
    plt.imshow(X_raw_like[idx], cmap='gray')
    plt.axis('off')
plt.show()

## Entrenamiento del modelo


In [ ]:
# --- División de datos ---
X_train, X_test, y_train, y_test = train_test_split(
    X_raw_like, y_custom,
    test_size=0.20,  # Porcentaje para pruebas
    random_state=42,
    stratify=y_custom # Mantener proporciones de cada clase
)

# --- Ajustamos las dimensiones  ---
if X_train.ndim == 3:
    X_train = X_train[...,None]
    X_test = X_test[..., None]
    print(f"Dimensiones finales Train: {X_train.shape}")
    print(f"Dimensiones finales Test: {X_test.shape}")

# --- Parámetros  ---
bs = 16 #tamaño
lr = 0.0001 #learning rate

# --- Mapeo de clases  ---
y_train = y_train.astype(int)
y_test = y_test.astype(int)

print("\n¡Datos listos para entrenar!")

## Construir arquitectura del modelo

In [ ]:
def build_model_from_reference(num_classes, input_shape=(28, 28, 1)):
    """
    Crea el modelo basado en el código de referencia.
    ¡CORREGIDO! Usando la API Funcional para que SÍ se pueda exportar.
    """
    
    # 1. Definir la entrada explícitamente (esto arregla el error de exportación)
    inputs = Input(shape=input_shape)

    # 2. Definir el resto de la arquitectura (idéntica a la tuya)
    x = Conv2D(32, 3, activation='relu')(inputs)
    x = MaxPool2D()(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPool2D()(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = MaxPool2D()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # 3. Crear el modelo
    model = Model(inputs=inputs, outputs=outputs) # Modelo Funcional
    
    # 4. Compilador (este ya está bien en tu código)
    optimizer1 = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer= optimizer1, loss='sparse_categorical_crossentropy' , metrics=['accuracy'])

    return model

##  Red B (Sin experiencia )


Entrenamos el modelo desde cero, usando **únicamente** nuestro dataset de 8 formas.

In [ ]:
print("--- Entrenando Red B (Solo Dataset Propio) ---")

#  Construir
model_B = build_model_from_reference(
    num_classes=NUM_CLASSES_CUSTOM, # 8 clases
    input_shape=INPUT_SHAPE
)

model_B.summary()

#  Entrenar
history_B = model_B.fit(
    X_train, y_train,
    epochs=50, # 50 épocas para darle una oportunidad
    batch_size=bs,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\nEntrenamiento de Red B finalizado.")

## Red A (Con Experiencia)

Dividido en 3 partes:
1.   Cargar y Pre-entrenar un modelo con **MNIST**.
2.   Congelar ese modelo y **Modificar** la última capa.
3.   Re-entrenar (ajustar) con nuestro dataset de 8 formas.

In [ ]:
# --- Cargar y Entrenar con MNIST  ---
print("Cargando y preparando dataset MNIST...")
(mnist_train_X, mnist_train_y), (mnist_test_X, mnist_test_y) = mnist.load_data()

# Pre-procesamiento de MNIST
mnist_train_X = mnist_train_X / 255.0
mnist_test_X = mnist_test_X / 255.0
mnist_train_X = mnist_train_X[..., None]
mnist_test_X = mnist_test_X[..., None]
mnist_train_y = mnist_train_y.astype(int)
mnist_test_y = mnist_test_y.astype(int)

print(f"MNIST Train shape: {mnist_train_X.shape}")

# --- Construir el modelo base de MNIST ---
print("\n--- Pre-entrenando Red A (Base MNIST) ---")
model_A_base = build_model_from_reference(
    num_classes=10, # 10 clases para MNIST (0-9)
    input_shape=INPUT_SHAPE
)

print("Summary del modelo base de MNIST:")
model_A_base.summary()

# Entrenamos el modelo base
history_base = model_A_base.fit(
    mnist_train_X, mnist_train_y,
    epochs=5, # 5 épocas en MNIST es suficiente para aprender formas
    batch_size=bs,
    validation_data=(mnist_test_X, mnist_test_y),
    verbose=1
)

print("\nPre-entrenamiento de Red A finalizado.")

In [ ]:
# --- Congelar y Modificar la Última Capa ---
print("--- Congelando y Modificando Red A ---")

# 1. Congelamos las capas "base" (convolucionales)
print("Congelando capas convolucionales de la base de MNIST...")
for layer in model_A_base.layers:
    # Congelamos todo HASTA la capa Flatten
    if not isinstance(layer, (Flatten, Dense)):
        layer.trainable = False
        # print(f"CONGELANDO capa: {layer.name}") # Descomenta si quieres ver
    else:
        # Dejamos la "cabeza" (Flatten y Dense) ENTRENABLE
        layer.trainable = True
        # print(f"ENTRENABLE capa: {layer.name}") # Descomenta si quieres ver

# 2. Creamos el nuevo modelo (Red A)
base_input = model_A_base.input # Tomar la entrada de la base (AHORA SÍ FUNCIONA)
base_output = model_A_base.layers[-2].output # Tomar la salida de la penúltima capa (Dense 128)

# 3. Añadir nueva capa final
final_output = Dense(NUM_CLASSES_CUSTOM, activation='softmax', name='output_formas')(base_output)

# 4. Crear el modelo
model_A = Model(inputs=base_input, outputs=final_output)

# 5. Compilar (usando el lr de referencia)
optimizer_A = tf.keras.optimizers.Adam(learning_rate=lr)
model_A.compile(optimizer=optimizer_A, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\n--- Summary de Red A ---")
model_A.summary()

In [ ]:
# --- Re-entrenar (Ajustar) la Red A ---
print("\n--- Ajustando Red A (Solo Dataset Propio) ---")

# Entrenamos el modelo A (híbrido) solo con nuestro dataset de formas
# Keras solo actualizará los pesos de la última capa (Dense(8))
history_A = model_A.fit(
    X_train, y_train,
    epochs=50, # Mismas 50 épocas que Red B
    batch_size=bs,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\nAjuste de Red A finalizado.")

## Comparando las 2 Redes Neuronales

Evaluamos `model_A` y `model_B` en el **mismo conjunto de prueba** (`X_test`, `y_test` de nuestras 8 formas).

In [ ]:
# Gráficas de Entrenamiento (Accuracy y Loss)
plt.figure(figsize=(20, 8))

# Gráfica de Accuracy
plt.subplot(1, 2, 1)
plt.title('Validation Accuracy (Red A vs. Red B)')
plt.plot(history_B.history['val_accuracy'], label='Red B (Desde Cero)', color='blue', linestyle='--')
plt.plot(history_A.history['val_accuracy'], label='Red A (Transfer MNIST)', color='red', linestyle='-')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()

# Gráfica de Pérdida (Loss)
plt.subplot(1, 2, 2)
plt.title('Validation Loss (Red A vs. Red B)')
plt.plot(history_B.history['val_loss'], label='Red B (Desde Cero)', color='blue', linestyle='--')
plt.plot(history_A.history['val_loss'], label='Red A (Transfer MNIST)', color='red', linestyle='-')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Reporte de Clasificación y Accuracy Final
print("Evaluando modelos en el conjunto de prueba (tus 8 formas)...")

# Evaluación general
score_B = model_B.evaluate(X_test, y_test, verbose=0)
score_A = model_A.evaluate(X_test, y_test, verbose=0)

print("\n--- Resultados Finales ---")
print(f"Red B (Desde Cero):   Accuracy = {score_B[1]*100:.2f}% | Loss = {score_B[0]:.4f}")
print(f"Red A (Transfer MNIST): Accuracy = {score_A[1]*100:.2f}% | Loss = {score_A[0]:.4f}")


# Obtener predicciones (enteros) para el reporte
y_pred_B_probs = model_B.predict(X_test)
y_pred_B = np.argmax(y_pred_B_probs, axis=1)

y_pred_A_probs = model_A.predict(X_test)
y_pred_A = np.argmax(y_pred_A_probs, axis=1)

# Nombres de las clases para el reporte
target_names = list(class_names_map.values())

print("\n=========================================================")
print("           REPORTE DE CLASIFICACIÓN - RED B (Desde Cero)")
print("=========================================================")
print(classification_report(y_test, y_pred_B, target_names=target_names, zero_division=0))

print("\n=========================================================")
print("        REPORTE DE CLASIFICACIÓN - RED A (Transfer MNIST)")
print("=========================================================")
print(classification_report(y_test, y_pred_A, target_names=target_names, zero_division=0))

In [ ]:
# Matrices de Confusión
cm_B = confusion_matrix(y_test, y_pred_B)
cm_A = confusion_matrix(y_test, y_pred_A)

plt.figure(figsize=(22, 10))

plt.subplot(1, 2, 1)
sns.heatmap(cm_B, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Matriz de Confusión - Red B (Desde Cero)')
plt.xlabel('Predicción')
plt.ylabel('Real')

plt.subplot(1, 2, 2)
sns.heatmap(cm_A, annot=True, fmt='d', cmap='Reds',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Matriz de Confusión - Red A (Transfer MNIST)')
plt.xlabel('Predicción')
plt.ylabel('Real')

plt.show()

In [ ]:
# --- Tomamos una imagen de prueba  ---
idx = np.random.choice(X_test.shape[0], 1)[0]
im = X_test[idx] # Esta imagen ya es (28, 28, 1)
label_int = y_test[idx]
nombre_forma = class_names_map.get(label_int, 'Desconocida')

# Mostramos la imagen de prueba
plt.title(f'(Test) id:{idx} val:{label_int} ({nombre_forma})')
plt.axis('off')
plt.imshow(im.reshape(28, 28), cmap='gray') # reshape a (28,28) para mostrar
plt.show()


# --- Usando el modelo B  ---
salida_B = model_B.predict(im[None, ...])[0] # im[None,...] añade el batch_size
pred_B_int = salida_B.argmax()
pred_B_nombre = class_names_map.get(pred_B_int, '???')

plt.figure(figsize=(10,4))
plt.title(f"Red B: Predicción: {pred_B_nombre} ({pred_B_int}), Label: {nombre_forma} ({label_int})")
plt.ylabel("Probabilidad")
plt.xlabel("Clase")
plt.ylim([0, 1])
plt.bar(np.arange(len(salida_B)), salida_B)
plt.xticks(np.arange(len(salida_B)), labels=[name[0:3] for name in target_names])
plt.show()


# --- Usando el modelo A  ---
salida_A = model_A.predict(im[None, ...])[0]
pred_A_int = salida_A.argmax()
pred_A_nombre = class_names_map.get(pred_A_int, '???')

plt.figure(figsize=(10,4))
plt.title(f"Red A: Predicción: {pred_A_nombre} ({pred_A_int}), Label: {nombre_forma} ({label_int})")
plt.ylabel("Probabilidad")
plt.xlabel("Clase")
plt.ylim([0, 1])
plt.bar(np.arange(len(salida_A)), salida_A, color='red')
plt.xticks(np.arange(len(salida_A)), labels=[name[0:3] for name in target_names])
plt.show()

In [ ]:
import tensorflowjs as tfjs
import os
import shutil

print("Paso 1: Instalando tensorflowjs (puede tardar un momento)...")
print("¡Instalado!")

# --- EXPORTAR RED B ---
print("\nPaso 2: Exportando Red B...")
MODEL_B_DIR = 'model_B_web'
MODEL_B_H5 = 'model_B.h5'
model_B.save(MODEL_B_H5) # Guardar en H5
os.makedirs(MODEL_B_DIR, exist_ok=True) # Crear carpeta de salida
!tensorflowjs_converter --input_format=keras {MODEL_B_H5} {MODEL_B_DIR} # Convertir

# --- EXPORTAR RED A ---
print("\nPaso 3: Exportando Red A...")
MODEL_A_DIR = 'model_A_web'
MODEL_A_H5 = 'model_A.h5'
model_A.save(MODEL_A_H5) # Guardar en H5
os.makedirs(MODEL_A_DIR, exist_ok=True) # Crear carpeta de salida
!tensorflowjs_converter --input_format=keras {MODEL_A_H5} {MODEL_A_DIR} # Convertir

# --- VERIFICAR Y COMPRIMIR ---
print("\nPaso 4: Verificando archivos...")
if os.path.exists(f'{MODEL_B_DIR}/model.json') and os.path.exists(f'{MODEL_A_DIR}/model.json'):
    print("¡ÉXITO! Los archivos .json fueron creados.")
    print("Comprimiendo carpetas en zips...")
    
    # Comprimir cada carpeta web en su propio zip (más fácil de manejar)
    shutil.make_archive('model_B_web', 'zip', MODEL_B_DIR)
    shutil.make_archive('model_A_web', 'zip', MODEL_A_DIR)
    
    print("\n¡HECHO! Descarga 'model_A_web.zip' y 'model_B_web.zip' desde el panel de archivos.")
    print("Sube el contenido de esos zips a tu GitHub Pages.")
else:
    print("\n¡ERROR! La conversión falló. Los archivos .json no se crearon.")
    print("Asegúrate de haber reemplazado las celdas 7 y 12 como se indicó.")

## Conclusión
La red B obtuvo una *mejor precisión* que la red A. Esto podría deberse a
- Dataset suficientemente grande: Alrededor de 2000 imágenes
- El conocimiento de MNIST (números) no ayudó para formas geométricas. El transfer learning solo es útil cuando los objetos a detectar son similares
